# Agentic RAG

Agent是一个使用LLM作为engine的智能体系统，它具备访问外部工具的能力。所以系统核心有2点， 引擎（LLM） & 工具（tool）。引擎主要起到规划和反思的作用，工具让大模型可以去做很多复杂的事情，比如生成图片，生成视频，文档问答等等。

Agentic RAG的概念，在2024年，Q2左右属于比较热门的一个类型，本质上是将RAG的召回看作是一个工具。由引擎来决策应该何时调用召回工具，以及调用工具的参数（问题拆解）是什么。


这里用到的Agent框架是huggingface的transformers仓库自带的agent功能。

llm_engine使用的智谱的模型，当然可以也可以自己预定义，例如下面使用Qwen来作为engine。

```python
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" 

model = AutoModelForCausalLM.from_pretrained(
    "./Qwen1.5-14B-Chat",
    torch_dtype=torch.bfloat16,
    device_map="auto"
).eval()

def llm_engine(messages, stop_sequences=["<im_end>"]):
    # print(messages)
    messages[1]['content'] = messages[0]['content'] + '\n'  + messages[1]['content']
    messages[0]['content'] = "You are a helpful assistant."
    
    print(messages)
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    
    if stop_sequences:
        stop_token_ids = [tokenizer(x, return_tensors='pt', add_special_tokens=False)['input_ids'] for x in stop_sequences]
        stop_token_ids = [torch.LongTensor(x).to('cuda') for x in stop_token_ids]
        stopping_criteria = [StopOnTokens(stop_token_ids)]
    else:
        stopping_criteria = []
        
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
        do_sample=False,
        # stop = stop_sequences
        stopping_criteria = stopping_criteria
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response
```

In [16]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key=".WvLXD1MsFsGklOZa") # 填写您自己的APIKey

def llm_engine(messages, stop_sequences=None):
    # print(messages)
    for message in messages:
        if message['role'] == 'tool-response':
            message['role'] = 'user'
    response = client.chat.completions.create(
        model="glm-4-plus", 
        messages=messages,
        stop=stop_sequences
    )
    return response.choices[0].message.content


from transformers import Tool

class TextRetrieval(Tool):
    name = "text_retrieval"
    description = (
        "这是一个根据根据用户问题搜索文档库的工具，它返回一个跟用户问题相关的文档片段"
    )

    inputs = {
        "query": {
            "type": "text",
            "description": "用户问题",
        }
    }
    output_type = "text"

    def forward(self, query):
        chunk1 = """西湖
西湖，又名钱塘湖，位于中国浙江省杭州市西湖区龙井路1号，杭州市区西部，汇水面积为21.22平方千米，湖面面积为6.38平方千米，为江南三大名湖之一。 [4-5] [22]
西湖南、西、北三面环山，东邻城区，南部和钱塘江隔山相邻，湖中白堤、苏堤、杨公堤、赵公堤将湖面分割成若干水面，湖中有三岛，西湖群山以西湖为中心，由近及远可分为四个层次。 [7]西湖流域内年径流量为1400万立方米，蓄水量近1400万立方米。 [9]西湖是一座设备齐全、管理完善的灌溉济运水库，通过河渠和闸门，为杭州市区及周边农田提供了充足的水源。西湖还可以协助调节钱塘江的水位，防止城市内涝。 [16]
2007年，西湖所在的杭州西湖风景名胜区被评为“国家AAAAA级旅游景区”。 [2] 2011年6月24日，杭州西湖文化景观被列入《世界遗产名录》。 [19]厉声教先生曾留下著名词篇《采桑子·西湖四咏》描写杭州西湖四季美景。"""

        chunk2 = """东湖
东湖，又称裹脚湖，长江右岸湖泊。位于湖北省武汉市城区东部。全湖面积在正常高水位19.78米时为31.75平方公里，湖泊容积8150万立米，如以水位21.0米计则为32.8平方公里，相应最大湖容为1.241亿立方米。 [1]
东湖是武汉市的重要水源地、水产养殖基地，也是知名生态旅游风景区、首批国家级风景名胜区。
东湖自古就是游览胜地，历史上屈原、李白等不少名人曾在东湖留下足迹。东湖是最大的楚文化游览中心，楚风浓郁，楚韵精妙，行吟阁名播遐迩，离骚碑誉为“三绝”，楚天台气势磅礴，楚才园名人荟萃，楚市、屈原塑像、屈原纪念馆，内涵丰富。也是解放后毛泽东同志除中南海外居住时间最长的地方。
东湖也是武汉市区重要的文化中心，周边聚集了武汉大学、华中科技大学、中国地质大学（武汉）等26所高等院校，中科院武汉植物园等56个国家、省、部属科研院所，东湖新技术开发区国家光电子产业基地——中国光谷、湖北省博物馆，湖北美术馆，文化底蕴深厚。 [2"""

        if '东湖' in query:
            return chunk2
        return chunk1
                

In [17]:
from transformers import Tool, load_tool, CodeAgent,ReactJsonAgent

In [19]:
agent = ReactJsonAgent(tools=[TextRetrieval()], verbose=1, llm_engine=llm_engine)

agent.run(
    "武汉东湖跟杭州西湖哪个大？，大多少呢？"
)

======== New task ========
武汉东湖跟杭州西湖哪个大？，大多少呢？
===== Calling LLM with this last message: =====
{'role': <MessageRole.USER: 'user'>, 'content': 'Task: 武汉东湖跟杭州西湖哪个大？，大多少呢？'}
Calling tool: 'text_retrieval' with arguments: {'query': '武汉东湖面积'}
东湖
东湖，又称裹脚湖，长江右岸湖泊。位于湖北省武汉市城区东部。全湖面积在正常高水位19.78米时为31.75平方公里，湖泊容积8150万立米，如以水位21.0米计则为32.8平方公里，相应最大湖容为1.241亿立方米。 [1]
东湖是武汉市的重要水源地、水产养殖基地，也是知名生态旅游风景区、首批国家级风景名胜区。
东湖自古就是游览胜地，历史上屈原、李白等不少名人曾在东湖留下足迹。东湖是最大的楚文化游览中心，楚风浓郁，楚韵精妙，行吟阁名播遐迩，离骚碑誉为“三绝”，楚天台气势磅礴，楚才园名人荟萃，楚市、屈原塑像、屈原纪念馆，内涵丰富。也是解放后毛泽东同志除中南海外居住时间最长的地方。
东湖也是武汉市区重要的文化中心，周边聚集了武汉大学、华中科技大学、中国地质大学（武汉）等26所高等院校，中科院武汉植物园等56个国家、省、部属科研院所，东湖新技术开发区国家光电子产业基地——中国光谷、湖北省博物馆，湖北美术馆，文化底蕴深厚。 [2
===== Calling LLM with this last message: =====
{'role': <MessageRole.TOOL_RESPONSE: 'tool-response'>, 'content': 'Observation: 东湖\n东湖，又称裹脚湖，长江右岸湖泊。位于湖北省武汉市城区东部。全湖面积在正常高水位19.78米时为31.75平方公里，湖泊容积8150万立米，如以水位21.0米计则为32.8平方公里，相应最大湖容为1.241亿立方米。 [1]\n东湖是武汉市的重要水源地、水产养殖基地，也是知名生态旅游风景区、首批国家级风景名胜区。\n东湖自古就是游览胜地，历史上屈原、李白等不少名人曾在东湖留下足迹。东湖是最大的楚文化游览中心，楚风浓郁，

'武汉东湖比杭州西湖大，面积差为31.75 - 6.38 = 25.37平方公里'

In [20]:
from transformers import Tool, load_tool, CodeAgent,ReactCodeAgent

In [21]:
agent = ReactCodeAgent(tools=[TextRetrieval()], verbose=1, llm_engine=llm_engine)

agent.run(
    "武汉东湖跟杭州西湖哪个大？，大多少呢？"
)

======== New task ========
武汉东湖跟杭州西湖哪个大？，大多少呢？
===== Calling LLM with these last messages: =====
[{'role': <MessageRole.SYSTEM: 'system'>, 'content': 'You will be given a task to solve as best you can.\nYou have access to the following tools:\n\n- text_retrieval: 这是一个根据根据用户问题搜索文档库的工具，它返回一个跟用户问题相关的文档片段\n    Takes inputs: {\'query\': {\'type\': \'text\', \'description\': \'用户问题\'}}\n\n- final_answer: Provides a final answer to the given problem\n    Takes inputs: {\'answer\': {\'type\': \'text\', \'description\': \'The final answer to the problem\'}}\n\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \'Thought:\', \'Code:\', and \'Observation:\' sequences.\n\nAt each step, in the \'Thought:\' sequence, you should first explain your reasoning towards solving the task, then the tools that you want to use.\nThen in the \'Code:\' sequence, you shold write the code in simple Python. The code sequence must end with \'/End code\' sequence.\nDuring each in

'武汉东湖比杭州西湖大，大 25.37 平方千米。'